In [1]:
import os
!pwd

/home/lok/Documents/ML_Projects/rice_leaf_disease/research


In [2]:
os.chdir('../')
!pwd

/home/lok/Documents/ML_Projects/rice_leaf_disease


In [3]:
from pathlib import Path
from typing import List
from dataclasses import dataclass


In [4]:
@dataclass(frozen=True)
class PrepareModelConfig:
    # Model Preparation Paths
    root_dir: Path
    model_path: Path
    
    # Model Architecture Params
    input_shape: tuple
    num_classes: int
    
    # Convolutional Layer Params
    conv1_filters: int
    conv1_kernel_size: int
    conv1_activation: str
    conv1_dropout_rate: float
    
    conv2_filters: int
    conv2_kernel_size: int
    conv2_activation: str
    conv2_dropout_rate: float
    
    conv3_filters: int
    conv3_kernel_size: int
    conv3_activation: str
    conv3_dropout_rate: float
    
    # Dense Layer Params
    dense1_units: int
    dense1_activation: str
    
    dense2_units: int
    dense2_activation: str
    
    # Training Params
    optimizer: str
    loss_function: str
    metrics: list


In [5]:
from rice_leaf_disease.constants import *
from rice_leaf_disease.utils.common import create_directories, read_yaml


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)  # Read config YAML
        self.params = read_yaml(params_filepath)  # Read params YAML
        
        # Ensure required directories are created
        create_directories([self.config.artifacts_root])

    def get_prepare_model_config(self) -> PrepareModelConfig:
        config = self.config.prepare_model  # Access prepare_model section in config
        create_directories([config.root_dir])  # Create root directory if it doesn't exist
        
        # Extracting parameters from params.yaml
        model_params = self.params.model_params
        
        prepare_model_config = PrepareModelConfig(
            # Paths from config
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),

            # Model architecture parameters from params.yaml
            input_shape=tuple(model_params.input_shape),
            num_classes=model_params.num_classes,
            
            # Convolutional layer parameters
            conv1_filters=model_params.layers.conv1.filters,
            conv1_kernel_size=model_params.layers.conv1.kernel_size,
            conv1_activation=model_params.layers.conv1.activation,
            conv1_dropout_rate=model_params.layers.conv1.dropout_rate,
            
            conv2_filters=model_params.layers.conv2.filters,
            conv2_kernel_size=model_params.layers.conv2.kernel_size,
            conv2_activation=model_params.layers.conv2.activation,
            conv2_dropout_rate=model_params.layers.conv2.dropout_rate,
            
            conv3_filters=model_params.layers.conv3.filters,
            conv3_kernel_size=model_params.layers.conv3.kernel_size,
            conv3_activation=model_params.layers.conv3.activation,
            conv3_dropout_rate=model_params.layers.conv3.dropout_rate,
            
            # Dense layer parameters
            dense1_units=model_params.layers.dense1.units,
            dense1_activation=model_params.layers.dense1.activation,
            
            dense2_units=model_params.layers.dense2.units,
            dense2_activation=model_params.layers.dense2.activation,
            
            # Training parameters
            optimizer=model_params.optimizer,
            loss_function=model_params.loss_function,
            metrics=model_params.metrics
        )
        
        return prepare_model_config



In [6]:
from tensorflow.keras import layers, models
import tensorflow as tf

2024-12-24 15:16:47.358785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-24 15:16:49.649794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2024-12-24 15:16:51,279: INFO : utils : NumExpr defaulting to 8 threads.]


In [7]:

class PrepareModel:
    def __init__(self, config: PrepareModelConfig):
        self.config = config
        self.model = None  # Initialize model as None, it will be built later.

    def get_model(self):
        # Build the model based on the config parameters
        self.model = self._build_custom_model()

        # Save the model (the path is defined in the config)
        self.save_model(path=self.config.model_path, model=self.model)

    def _build_custom_model(self):
        model = models.Sequential()

        # Add the input layer
        model.add(layers.Input(shape=self.config.input_shape))

        # First convolutional block
        model.add(layers.Conv2D(
            filters=self.config.conv1_filters,
            kernel_size=self.config.conv1_kernel_size,
            activation=self.config.conv1_activation,
            padding='same'
        ))
        model.add(layers.MaxPooling2D())
        model.add(layers.Dropout(self.config.conv1_dropout_rate))

        # Second convolutional block
        model.add(layers.Conv2D(
            filters=self.config.conv2_filters,
            kernel_size=self.config.conv2_kernel_size,
            activation=self.config.conv2_activation,
            padding='same'
        ))
        model.add(layers.MaxPooling2D())
        model.add(layers.Dropout(self.config.conv2_dropout_rate))

        # Third convolutional block
        model.add(layers.Conv2D(
            filters=self.config.conv3_filters,
            kernel_size=self.config.conv3_kernel_size,
            activation=self.config.conv3_activation,
            padding='same'
        ))
        model.add(layers.MaxPooling2D())

        # Flatten the output from convolutional layers
        model.add(layers.Flatten())

        # Fully connected (dense) layers
        model.add(layers.Dense(
            units=self.config.dense1_units,
            activation=self.config.dense1_activation
        ))

        # Output layer
        model.add(layers.Dense(
            units=self.config.num_classes,
            activation=self.config.dense2_activation
        ))

        # Compile the model with the specified optimizer, loss, and metrics
        model.compile(
            optimizer=self.config.optimizer,
            loss=self.config.loss_function,
            metrics=self.config.metrics
        )
        
        model.summary()

        return model



    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the model to the specified path
        model.save(path)


In [8]:
try:
    config = ConfigurationManager()
    prepare_model_config = config.get_prepare_model_config()
    prepare_model = PrepareModel(config=prepare_model_config)
    prepare_model.get_model()

except Exception as e:
    raise e

[2024-12-24 15:16:52,008: INFO : common : YAML file: config/config.yaml loaded successfully!]
[2024-12-24 15:16:52,017: INFO : common : YAML file: params.yaml loaded successfully!]
[2024-12-24 15:16:52,019: INFO : common : created directory at: artifacts]
[2024-12-24 15:16:52,020: INFO : common : created directory at: artifacts/prepare_model]


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,989,027 (15.22 MB)

 Trainable params: 3,989,027 (15.22 MB)

 Non-trainable params: 0 (0.00 B)